Импорт необходимых библиотек



In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
import matplotlib.pyplot as plt
from google.colab import files

Запись в массив исходных данных, полученных из публичного датасета

In [ ]:
train = tfds.load("cats_vs_dogs", split=['train[:100%]'], as_supervised=True)

Проверка вывода изображения и его метаинформации



In [ ]:
for img, metavalue in train[0].take(2):
  plt.figure()
  plt.imshow(img)
  print(metavalue)

Необходимо изменить разрешение картинки, для введения разрешения как константы для нейросети, что исключит возможные ошибки при обучении. Также размер 224х224 является чем-то средним между распознаваемой картинкой и совсем нераспознаваемой, что позволит получить от нейросети определенный результат и при этом сократить время обучения и обработки изображений

In [ ]:
size = 224 #нейросеть работает именно с разрешением 224х224
def image_resizing(img, metavalue):
  img = tf.cast(img, tf.float32)
  img = tf.image.resize(img, (size, size))
  img = img / 255.0 #для введения изображения в пикселях в диапазон от 0 до 1
  return img, metavalue

Применение функции изменения размера к учебному массиву и разбиение тренировочного массива на части, поскольку возможность обработки всего массива с графическими файлами напрямую зависит от системы пользователя  

In [ ]:
train_resized = train[0].map(image_resizing)
train_slices = train_resized.shuffle(1000).batch(16)

Импортирование слоевой структуры сверточной  нейросети и отказываемся от ее тренировки, поскольку алгоритм работы ей уже усвоен.

In [ ]:
base_layers = tf.keras.applications.MobileNetV2(input_shape=(size, size, 3), include_top = False)
base_layers.trainable = False

Создание модели

In [ ]:
model = tf.keras.Sequential([
                             base_layers,
                             GlobalAveragePooling2D(),
                             Dropout(0.2),
                             Dense(1)
])
model.compile(optimizer="adam", loss = tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics =['accuracy'])

Обучение нейронной сети

In [ ]:
model.fit(train_slices, epochs=1)

1454/1454 [==============================] - 853s 587ms/step - loss: 0.0673 - accuracy: 0.9740


Загрузка пользовательской картинки, данный блок может отличаться в зависимости от среды, которую используете

In [ ]:
files.upload()

Блок обработки пользовательского изображения и его вывод 

In [ ]:
new_image = load_img('34.jpg')
img_array = img_to_array(new_image)
img_resized, _ = image_resizing(img_array, _)
img_expanded = np.expand_dims(img_resized, axis = 0)
prediction = model.predict(img_expanded)
pred_label = "CAT" if prediction < 0.5 else "DOG"
plt.figure()
plt.imshow(new_image)
plt.title(f'{pred_label} {prediction}')

